# Day 11_3: 텍스트 마이닝 종합 프로젝트 (Text Mining Comprehensive Project)

## 📚 학습 목표

**Part 1: 프로젝트 파이프라인 설계**
1. 텍스트 마이닝 전체 워크플로우 이해
2. 데이터 로드 및 탐색적 분석 (EDA)
3. 텍스트 전처리 파이프라인 구축
4. 키워드 빈도 분석 및 TF-IDF 적용
5. 감성 분석 및 평점 예측

**Part 2: 시각화 및 인사이트 도출**
1. Plotly 인터랙티브 대시보드 구축
2. 워드클라우드 및 키워드 차트 생성
3. 카테고리/제품별 비교 분석
4. 비즈니스 의사결정 권장사항 도출

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 전체 파이프라인 | 프로젝트 재현성 확보 | 전처리→분석→시각화 자동화 |
| 감성 분석 | 고객 VOC 분석 | 긍정/부정 리뷰 자동 분류 |
| 키워드 추출 | 트렌드 파악 | 불만 요인, 인기 기능 발견 |
| 대시보드 | 경영진 보고 | 인터랙티브 리포트 제공 |
| 인사이트 도출 | 의사결정 지원 | 제품 개선 방향 제안 |

**분석가 관점**: 텍스트 마이닝 종합 프로젝트는 데이터 수집부터 비즈니스 권장사항까지 전체 분석 역량을 증명하는 포트폴리오의 핵심입니다.

---

## 프로젝트 개요: 전자제품 리뷰 분석

### 비즈니스 목표
전자제품 리뷰 데이터를 분석하여:
1. **고객 만족/불만 요인** 파악
2. **제품별/카테고리별 감성 트렌드** 분석
3. **개선이 필요한 제품** 식별
4. **비즈니스 의사결정 권장사항** 도출

### 데이터셋
- **파일**: `datasets/product_reviews.csv`
- **레코드 수**: 505개 리뷰
- **기간**: 2024년 1월 ~ 2025년 6월
- **컬럼**: review_id, product_name, category, review_text, rating, review_date, helpful_votes

---

## Part 1: 프로젝트 파이프라인 설계

### 1.1 라이브러리 임포트 및 데이터 로드

In [ ]:
# 필수 라이브러리
import pandas as pd
import numpy as np
import re
from collections import Counter

# 시각화
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 텍스트 분석
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# 워드클라우드
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 한글 형태소 분석 (설치 필요시: pip install konlpy)
try:
    from konlpy.tag import Okt
    okt = Okt()
    USE_KONLPY = True
    print("KoNLPy 사용 가능")
except ImportError:
    USE_KONLPY = False
    print("KoNLPy 미설치 - 기본 토큰화 사용")

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료")

In [ ]:
# 데이터 로드
df = pd.read_csv('datasets/product_reviews.csv')

print(f"📊 데이터셋 정보")
print(f"- 전체 리뷰 수: {len(df):,}개")
print(f"- 컬럼: {list(df.columns)}")
print(f"\n📅 데이터 기간: {df['review_date'].min()} ~ {df['review_date'].max()}")
print(f"\n🏷️ 카테고리 수: {df['category'].nunique()}개")
print(f"📦 제품 수: {df['product_name'].nunique()}개")

In [ ]:
# 데이터 미리보기
print("📋 샘플 데이터:")
df.head()

---

### 1.2 탐색적 데이터 분석 (EDA)

In [ ]:
# 기본 통계
print("📊 기본 통계:")
print(f"- 평균 평점: {df['rating'].mean():.2f}점")
print(f"- 평점 분포: {df['rating'].value_counts().sort_index().to_dict()}")
print(f"- 평균 도움 투표: {df['helpful_votes'].mean():.1f}개")
print(f"- 리뷰 평균 길이: {df['review_text'].str.len().mean():.0f}자")

In [ ]:
# 평점 분포 시각화
rating_counts = df['rating'].value_counts().sort_index()

fig = px.bar(
    x=rating_counts.index,
    y=rating_counts.values,
    labels={'x': '평점', 'y': '리뷰 수'},
    title='평점 분포',
    color=rating_counts.values,
    color_continuous_scale='RdYlGn'
)
fig.update_layout(showlegend=False, height=400)
fig.show()

In [ ]:
# 카테고리별 평균 평점
category_stats = df.groupby('category').agg({
    'rating': ['mean', 'count'],
    'helpful_votes': 'sum'
}).round(2)
category_stats.columns = ['평균_평점', '리뷰_수', '총_도움_투표']
category_stats = category_stats.sort_values('평균_평점', ascending=False)

print("📊 카테고리별 통계:")
category_stats

In [ ]:
# 카테고리별 평점 분포 시각화
fig = px.box(
    df, x='category', y='rating',
    title='카테고리별 평점 분포',
    color='category'
)
fig.update_layout(showlegend=False, height=500, xaxis_tickangle=-45)
fig.show()

---

### 1.3 텍스트 전처리 파이프라인

In [ ]:
def preprocess_text(text):
    """텍스트 전처리 함수"""
    # 소문자 변환 (영문)
    text = text.lower()
    
    # 특수문자 제거 (한글, 영문, 숫자, 공백만 유지)
    text = re.sub(r'[^가-힣a-z0-9\s]', ' ', text)
    
    # 연속 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# 전처리 적용
df['cleaned_text'] = df['review_text'].apply(preprocess_text)

print("전처리 전후 비교:")
print(f"원본: {df['review_text'].iloc[0]}")
print(f"전처리: {df['cleaned_text'].iloc[0]}")

In [ ]:
# 한글 불용어 정의
STOPWORDS = {
    '이', '그', '저', '것', '수', '등', '및', '더', '또', '때', '중',
    '좀', '잘', '많이', '정말', '진짜', '너무', '아주', '매우', '완전',
    '하고', '하면', '해서', '하는', '하지', '하게', '했어요', '합니다',
    '있어요', '없어요', '있는', '없는', '같아요', '같습니다',
    '이에요', '예요', '입니다', '습니다', '에요', '어요', '요',
    '의', '을', '를', '에', '와', '과', '로', '으로', '이나', '나',
    '은', '는', '가', '도', '만', '까지', '부터', '에서'
}

def tokenize_korean(text):
    """한글 토큰화 (명사 추출)"""
    if USE_KONLPY:
        # KoNLPy 사용
        nouns = okt.nouns(text)
    else:
        # 기본 공백 기반 토큰화
        nouns = text.split()
    
    # 불용어 제거 및 2글자 이상만
    tokens = [word for word in nouns if word not in STOPWORDS and len(word) >= 2]
    return tokens

# 토큰화 적용
df['tokens'] = df['cleaned_text'].apply(tokenize_korean)

print("토큰화 예시:")
print(f"텍스트: {df['cleaned_text'].iloc[0]}")
print(f"토큰: {df['tokens'].iloc[0]}")

---

### 1.4 키워드 빈도 분석

In [ ]:
# 전체 키워드 빈도
all_tokens = [token for tokens in df['tokens'] for token in tokens]
word_freq = Counter(all_tokens)

# 상위 20개 키워드
top_20_words = word_freq.most_common(20)
print("🔤 전체 상위 20개 키워드:")
for word, count in top_20_words:
    print(f"  {word}: {count}회")

In [ ]:
# 키워드 빈도 시각화
words, counts = zip(*top_20_words)

fig = px.bar(
    x=list(counts)[::-1],
    y=list(words)[::-1],
    orientation='h',
    title='상위 20개 키워드 빈도',
    labels={'x': '빈도', 'y': '키워드'},
    color=list(counts)[::-1],
    color_continuous_scale='Blues'
)
fig.update_layout(height=600, showlegend=False)
fig.show()

In [ ]:
# 긍정/부정 리뷰 키워드 비교
positive_df = df[df['rating'] >= 4]
negative_df = df[df['rating'] <= 2]

positive_tokens = [t for tokens in positive_df['tokens'] for t in tokens]
negative_tokens = [t for tokens in negative_df['tokens'] for t in tokens]

positive_freq = Counter(positive_tokens).most_common(15)
negative_freq = Counter(negative_tokens).most_common(15)

print("😊 긍정 리뷰 (4-5점) 상위 키워드:")
print([f"{w}({c})" for w, c in positive_freq])

print("\n😞 부정 리뷰 (1-2점) 상위 키워드:")
print([f"{w}({c})" for w, c in negative_freq])

---

### 1.5 TF-IDF 분석

In [ ]:
# TF-IDF 벡터화
df['tokens_str'] = df['tokens'].apply(lambda x: ' '.join(x))

tfidf = TfidfVectorizer(max_features=100)
tfidf_matrix = tfidf.fit_transform(df['tokens_str'])

# TF-IDF 점수 상위 키워드
feature_names = tfidf.get_feature_names_out()
tfidf_scores = tfidf_matrix.sum(axis=0).A1
tfidf_ranking = sorted(zip(feature_names, tfidf_scores), key=lambda x: x[1], reverse=True)

print("📈 TF-IDF 상위 20개 키워드:")
for word, score in tfidf_ranking[:20]:
    print(f"  {word}: {score:.2f}")

In [ ]:
# 카테고리별 TF-IDF 상위 키워드
def get_category_tfidf_keywords(category, top_n=10):
    """카테고리별 TF-IDF 상위 키워드 추출"""
    cat_df = df[df['category'] == category]
    if len(cat_df) < 5:
        return []
    
    cat_tfidf = TfidfVectorizer(max_features=50)
    cat_matrix = cat_tfidf.fit_transform(cat_df['tokens_str'])
    
    features = cat_tfidf.get_feature_names_out()
    scores = cat_matrix.sum(axis=0).A1
    
    return sorted(zip(features, scores), key=lambda x: x[1], reverse=True)[:top_n]

# 주요 카테고리 키워드
print("🏷️ 카테고리별 TF-IDF 상위 키워드:")
for cat in df['category'].unique()[:5]:
    keywords = get_category_tfidf_keywords(cat, 5)
    print(f"\n[{cat}]")
    print(f"  {[f'{w}({s:.1f})' for w, s in keywords]}")

---

### 1.6 감성 분석

In [ ]:
# 감성 사전 기반 분석
POSITIVE_WORDS = {
    '좋아요', '최고', '만족', '훌륭', '뛰어나', '편리', '깔끔', '빠르',
    '선명', '강력', '정확', '부드럽', '안정', '추천', '대박', '예뻐',
    '편해요', '좋습니다', '마음에', '최고입니다', '좋아', '좋은', '좋고'
}

NEGATIVE_WORDS = {
    '아쉬워', '불편', '실망', '느려', '비싸', '무거워', '시끄럽', '뜨거워',
    '부족', '불안', '약해', '어려워', '귀찮', '고장', '오류', '끊겨',
    '아쉽', '비쌉', '무겁', '불안정', '아쉬워요', '아쉽습니다', '불편합니다'
}

def calculate_sentiment_score(text):
    """감성 점수 계산 (-1 ~ 1)"""
    pos_count = sum(1 for word in POSITIVE_WORDS if word in text)
    neg_count = sum(1 for word in NEGATIVE_WORDS if word in text)
    
    total = pos_count + neg_count
    if total == 0:
        return 0
    return (pos_count - neg_count) / total

# 감성 점수 계산
df['sentiment_score'] = df['review_text'].apply(calculate_sentiment_score)

# 감성 라벨
df['sentiment'] = df['sentiment_score'].apply(
    lambda x: '긍정' if x > 0 else ('부정' if x < 0 else '중립')
)

print("📊 감성 분포:")
print(df['sentiment'].value_counts())

In [ ]:
# 감성 vs 평점 일치도
df['rating_sentiment'] = df['rating'].apply(
    lambda x: '긍정' if x >= 4 else ('부정' if x <= 2 else '중립')
)

match_rate = (df['sentiment'] == df['rating_sentiment']).mean()
print(f"감성-평점 일치율: {match_rate:.1%}")

# 교차표
cross_tab = pd.crosstab(df['rating_sentiment'], df['sentiment'], margins=True)
print("\n📊 감성-평점 교차표:")
cross_tab

---

## Part 2: 시각화 및 인사이트 도출

### 2.1 워드클라우드 생성

In [ ]:
# 워드클라우드 생성
all_text = ' '.join(df['tokens_str'])

# 한글 폰트 설정 (시스템에 따라 경로 변경 필요)
try:
    font_path = '/System/Library/Fonts/AppleGothic.ttf'  # Mac
    wordcloud = WordCloud(
        font_path=font_path,
        width=800,
        height=400,
        background_color='white',
        colormap='viridis',
        max_words=100
    ).generate(all_text)
    
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('전체 리뷰 워드클라우드', fontsize=16)
    plt.tight_layout()
    plt.show()
except:
    print("워드클라우드 생성 실패 - 폰트 경로 확인 필요")

---

### 2.2 Plotly 대시보드

In [ ]:
# 종합 대시보드 (2x2 레이아웃)
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('평점 분포', '카테고리별 평균 평점', '월별 리뷰 추이', '감성 분포'),
    specs=[[{"type": "bar"}, {"type": "bar"}],
           [{"type": "scatter"}, {"type": "pie"}]]
)

# 1) 평점 분포
rating_data = df['rating'].value_counts().sort_index()
fig.add_trace(
    go.Bar(x=rating_data.index, y=rating_data.values, marker_color='steelblue', name='평점'),
    row=1, col=1
)

# 2) 카테고리별 평균 평점
cat_rating = df.groupby('category')['rating'].mean().sort_values(ascending=True)
fig.add_trace(
    go.Bar(x=cat_rating.values, y=cat_rating.index, orientation='h', marker_color='coral', name='카테고리'),
    row=1, col=2
)

# 3) 월별 리뷰 추이
df['review_month'] = pd.to_datetime(df['review_date']).dt.to_period('M').astype(str)
monthly_counts = df.groupby('review_month').size()
fig.add_trace(
    go.Scatter(x=monthly_counts.index, y=monthly_counts.values, mode='lines+markers', name='월별'),
    row=2, col=1
)

# 4) 감성 분포
sentiment_counts = df['sentiment'].value_counts()
fig.add_trace(
    go.Pie(labels=sentiment_counts.index, values=sentiment_counts.values, name='감성'),
    row=2, col=2
)

fig.update_layout(height=800, title_text='전자제품 리뷰 분석 대시보드', showlegend=False)
fig.show()

---

### 2.3 제품별 상세 분석

In [ ]:
# 제품별 통계
product_stats = df.groupby('product_name').agg({
    'rating': ['mean', 'count', 'std'],
    'helpful_votes': 'sum',
    'sentiment_score': 'mean'
}).round(2)
product_stats.columns = ['평균평점', '리뷰수', '평점표준편차', '도움투표', '감성점수']

# 리뷰 10개 이상 제품만
product_stats = product_stats[product_stats['리뷰수'] >= 10]

print("📊 제품별 통계 (리뷰 10개 이상):")
product_stats.sort_values('평균평점', ascending=False)

In [ ]:
# 평점 vs 감성점수 산점도
fig = px.scatter(
    product_stats.reset_index(),
    x='평균평점',
    y='감성점수',
    size='리뷰수',
    hover_name='product_name',
    title='제품별 평점 vs 감성점수',
    labels={'평균평점': '평균 평점', '감성점수': '평균 감성 점수'}
)
fig.add_hline(y=0, line_dash="dash", line_color="gray")
fig.add_vline(x=3.5, line_dash="dash", line_color="gray")
fig.update_layout(height=500)
fig.show()

---

### 2.4 시간대별 트렌드 분석

In [ ]:
# 월별 평균 평점 추이
monthly_rating = df.groupby('review_month')['rating'].mean()

fig = px.line(
    x=monthly_rating.index,
    y=monthly_rating.values,
    title='월별 평균 평점 추이',
    labels={'x': '월', 'y': '평균 평점'},
    markers=True
)
fig.add_hline(y=monthly_rating.mean(), line_dash="dash", annotation_text="전체 평균")
fig.update_layout(height=400)
fig.show()

---

### 2.5 비즈니스 인사이트 및 권장사항

In [ ]:
# 인사이트 도출
print("="*60)
print("📊 텍스트 마이닝 분석 결과 요약")
print("="*60)

# 1. 전반적 만족도
avg_rating = df['rating'].mean()
positive_rate = (df['rating'] >= 4).mean()
print(f"\n1️⃣ 전반적 고객 만족도")
print(f"   - 평균 평점: {avg_rating:.2f}점")
print(f"   - 긍정 리뷰 비율 (4-5점): {positive_rate:.1%}")

# 2. 주요 긍정 키워드
print(f"\n2️⃣ 고객이 좋아하는 점 (긍정 키워드)")
for word, count in positive_freq[:5]:
    print(f"   - {word}: {count}회 언급")

# 3. 주요 불만 요인
print(f"\n3️⃣ 개선이 필요한 점 (부정 키워드)")
for word, count in negative_freq[:5]:
    print(f"   - {word}: {count}회 언급")

# 4. 주의가 필요한 제품
low_rating_products = product_stats[product_stats['평균평점'] < 3.5].sort_values('평균평점')
print(f"\n4️⃣ 주의 필요 제품 (평점 3.5 미만)")
if len(low_rating_products) > 0:
    for product in low_rating_products.index[:3]:
        rating = low_rating_products.loc[product, '평균평점']
        print(f"   - {product}: {rating}점")
else:
    print("   - 해당 제품 없음")

# 5. 권장사항
print(f"\n5️⃣ 비즈니스 권장사항")
print("   - 가격 경쟁력 강화: '비싸다' 키워드 빈출")
print("   - 배터리 성능 개선: '배터리' 관련 불만 다수")
print("   - AS 서비스 강화: '고장', 'AS' 관련 이슈 존재")
print("   - 제품 경량화 검토: '무겁다' 키워드 자주 등장")

print("\n" + "="*60)

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 데이터 로드 및 기본 통계 ⭐

**문제**: `datasets/product_reviews.csv` 파일을 로드하고, 다음 정보를 출력하세요:
- 전체 리뷰 수
- 고유 제품 수
- 평균 평점

**기대 결과**:
```
전체 리뷰 수: 505개
고유 제품 수: XX개
평균 평점: X.XX점
```

In [ ]:
# 여기에 코드를 작성하세요


---

### Q2. 텍스트 전처리 함수 구현 ⭐

**문제**: 텍스트에서 특수문자를 제거하고 소문자로 변환하는 `clean_text(text)` 함수를 구현하세요.

```python
sample = "화면이 정말 선명해요!! 최고입니다 :)"
```

**기대 결과**: `"화면이 정말 선명해요 최고입니다"`

In [ ]:
# 여기에 코드를 작성하세요


---

### Q3. 평점별 리뷰 수 계산 ⭐⭐

**문제**: 평점별 리뷰 수를 계산하고 Plotly 막대 그래프로 시각화하세요.

**기대 결과**: 1~5점까지 평점별 리뷰 수 막대 그래프

In [ ]:
# 여기에 코드를 작성하세요


---

### Q4. 카테고리별 평균 평점 계산 ⭐⭐

**문제**: 카테고리별 평균 평점을 계산하고, 가장 높은 평점과 가장 낮은 평점의 카테고리를 출력하세요.

**기대 결과**:
```
최고 평점 카테고리: XXX (X.XX점)
최저 평점 카테고리: XXX (X.XX점)
```

In [ ]:
# 여기에 코드를 작성하세요


---

### Q5. 키워드 빈도 분석 ⭐⭐⭐

**문제**: 전체 리뷰에서 가장 많이 등장하는 단어 상위 10개를 추출하세요. (공백 기준 토큰화, 2글자 이상)

**기대 결과**: 상위 10개 단어와 빈도

In [ ]:
# 여기에 코드를 작성하세요


---

### Q6. 긍정/부정 리뷰 비교 ⭐⭐⭐

**문제**: 긍정 리뷰(4-5점)와 부정 리뷰(1-2점)의 평균 리뷰 길이를 비교하세요.

**기대 결과**:
```
긍정 리뷰 평균 길이: XX자
부정 리뷰 평균 길이: XX자
```

In [ ]:
# 여기에 코드를 작성하세요


---

### Q7. TF-IDF로 중요 키워드 추출 ⭐⭐⭐

**문제**: `sklearn`의 `TfidfVectorizer`를 사용해 전체 리뷰에서 TF-IDF 점수 상위 10개 키워드를 추출하세요.

**기대 결과**: TF-IDF 상위 10개 키워드와 점수

In [ ]:
# 여기에 코드를 작성하세요


---

### Q8. 감성 분석 함수 구현 ⭐⭐⭐⭐

**문제**: 긍정/부정 단어 사전을 기반으로 감성 점수를 계산하는 함수를 구현하고, 전체 리뷰에 적용하세요.

```python
positive_words = {'좋아요', '최고', '만족', '훌륭', '추천'}
negative_words = {'별로', '실망', '불편', '아쉬워', '비싸'}
```

**기대 결과**: 감성 분포 (긍정/부정/중립 리뷰 수)

In [ ]:
# 여기에 코드를 작성하세요


---

### Q9. 제품별 리뷰 분석 대시보드 ⭐⭐⭐⭐

**문제**: 특정 제품(예: '삼성 갤럭시 S24')의 리뷰를 분석하는 대시보드를 만드세요:
1. 해당 제품 리뷰만 필터링
2. 평점 분포 파이 차트
3. 월별 리뷰 수 추이

**기대 결과**: 2개 서브플롯이 있는 대시보드

In [ ]:
# 여기에 코드를 작성하세요


---

### Q10. 종합 인사이트 리포트 생성 ⭐⭐⭐⭐⭐

**문제**: 전체 데이터를 분석하여 다음 내용을 포함하는 종합 리포트를 생성하세요:
1. 전체 데이터 요약 통계
2. 가장 만족도가 높은/낮은 카테고리 (리뷰 10개 이상)
3. 자주 언급되는 긍정/부정 키워드 각 5개
4. 비즈니스 권장사항 3가지

**기대 결과**: 체계적인 분석 리포트 출력

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 프로젝트 파이프라인 핵심 요약

| 단계 | 주요 작업 | 핵심 함수/라이브러리 |
|------|----------|--------------------|
| 데이터 로드 | CSV 읽기, 기본 EDA | `pd.read_csv()`, `.describe()` |
| 전처리 | 특수문자 제거, 토큰화 | `re.sub()`, `str.split()` |
| 키워드 분석 | 빈도 계산, TF-IDF | `Counter`, `TfidfVectorizer` |
| 감성 분석 | 사전 기반 감성 점수 | 커스텀 함수 |
| 시각화 | 차트, 워드클라우드 | `plotly`, `WordCloud` |

### Part 2: 시각화 및 인사이트 핵심 요약

| 시각화 유형 | 활용 목적 | Plotly 함수 |
|------------|----------|------------|
| 막대 그래프 | 평점/키워드 빈도 | `px.bar()`, `go.Bar()` |
| 파이 차트 | 비율 분포 | `go.Pie()` |
| 선 그래프 | 시간 추이 | `px.line()`, `go.Scatter()` |
| 산점도 | 변수 관계 | `px.scatter()` |
| 서브플롯 | 종합 대시보드 | `make_subplots()` |

### 💡 실무 팁

1. **파이프라인 자동화**: 전처리→분석→시각화를 함수로 모듈화하여 재사용성 확보
2. **불용어 커스터마이징**: 도메인별로 불용어 사전을 직접 구축하면 분석 품질 향상
3. **감성 사전 확장**: 기본 사전 외에 업종별 특화 긍정/부정 단어 추가
4. **대시보드 스토리텔링**: 데이터 → 인사이트 → 권장사항 순서로 구성
5. **정량+정성 결합**: 숫자 통계와 함께 실제 리뷰 예시를 제시하면 설득력 증가

---

**프로젝트 완료**: 텍스트 마이닝의 전체 파이프라인을 실습했습니다. 이 패턴을 다른 텍스트 데이터(뉴스, SNS, 고객 상담 등)에도 적용해보세요!